# Opertura d'una nova oficina i predicció de potencials localitzacions futures
 

|   INTEGRANTS                    | Arnau Muñoz Barrera     | Míriam López Sánchez       | Luis Martínez Zamora       | Marc Rodríguez Vitolo
| -------------------------- | ----------------------- | -------------------------- | -------------------------- | --------------------------

### 1. Objectius

L'objectiu d'aquest projecte es centra en buscar ubicacions idònies per obrir noves oficines de la Caixa d'Enginyers, valorant tant la possiblitat d'oficines fixes com oficinse mòbils que arriben a una determinada zona. A més, es desenvoluparà també un model predictiu amb l'ajuda d'IA que permeti identificar potencials localitzacions futures.

### 2. Font de les dades

Com a dades per analitzar en aquest projecte s'han agafat conjunts de dades extrets de fonts oficials: INE, BdE i propis de Caixa d'Enginyers.

### 3. Metodologia seguida

- Recerca de dades
- Anàlisi de dades amb python (realització de gràfiques i extracció d'estadístiques)
- Interpretació de resultats
- Elecció de les millors ubicacions posibles
- Entrenament del model predictiu
- Test del model predictiu
- Extracció de conclusions


### 4. Anàlisi de les dades
##### 1. EDA (exploratory data analysis)
Com a primer pas, caldrà que importar totes les llibreries i utilitats que es faran servir en el desenvolupament del projecte:


In [ ]:
# Import libraries 

%pip install pandas seaborn scikit-learn category_encoders scikit-optimize scipy openpyxl

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import gist_heat_r
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm

import warnings
warnings.simplefilter(action='ignore')

A continuació, es farà la importació dels datasets (arxius d'extensió **.xlsx**) a analitzar i s'analitzaran breument:

In [ ]:
dataframe_bancs_per_provincia = pd.read_excel('data/Bancs per provincia.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_bancs_per_provincia: ", dataframe_bancs_per_provincia.shape)

# View complete dataframe with print(dataframe_bancs_per_provincia)
print(dataframe_bancs_per_provincia.head(5))

In [ ]:
dataframe_empresa_per_mida_and_provincia = pd.read_excel('data/Empresa per mida y provincia 2008-2022.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_empresa_per_mida_and_provincia: ", dataframe_empresa_per_mida_and_provincia.shape)

print(dataframe_empresa_per_mida_and_provincia.head(5))

In [ ]:
df = dataframe_empresa_per_mida_and_provincia.copy()

prov_col = 'Provincias' if 'Provincias' in df.columns else 'Provincia'

df = df[
    (df['CCAA'].str.strip().str.lower() != 'total') &      
    (df[prov_col].astype(str).str.strip().str.lower() == 'total')
]

columnas_total = [col for col in df.columns if ('Total' in col and col.split()[0].isdigit())]
columnas_total = sorted(columnas_total, key=lambda c: int(c.split()[0]))

df_grouped = df.groupby('CCAA', as_index=False)[columnas_total].sum()

plt.figure(figsize=(14, 7))

comunidades = df_grouped['CCAA'].values
num_comunidades = len(comunidades)
colors = cm.get_cmap('tab20', num_comunidades)

años = [int(col.split()[0]) for col in columnas_total]

for i, row in df_grouped.iterrows():
    comunidad = row['CCAA']
    valores = row[columnas_total].values.astype(float)
    plt.plot(años, valores, marker='o', color=colors(i), label=comunidad, linewidth=2)

plt.xlabel('Año', fontsize=12)
plt.ylabel('Número de Empresas', fontsize=12)
plt.title('Evolución del Número de Empresas por Comunidad Autónoma en España', fontsize=14)
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.')))
plt.legend(title='Comunidad Autónoma', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df = dataframe_empresa_per_mida_and_provincia.copy()

prov_col = 'Provincias' if 'Provincias' in df.columns else 'Provincia'

provincias_catalunya = ['Barcelona', 'Girona', 'Lleida', 'Tarragona']
df_cat = df[df[prov_col].isin(provincias_catalunya)]

columnas_total = [col for col in df_cat.columns if ('Total' in col and col.split()[0].isdigit())]
columnas_total = sorted(columnas_total, key=lambda c: int(c.split()[0]))
años = [int(col.split()[0]) for col in columnas_total]

plt.figure(figsize=(12, 6))

colors = cm.get_cmap('tab10', len(provincias_catalunya))

for i, prov in enumerate(provincias_catalunya):
    datos_prov = df_cat[df_cat[prov_col] == prov]
    valores = datos_prov[columnas_total].values.flatten().astype(float)
    plt.plot(años, valores, marker='o', linewidth=2, color=colors(i), label=prov)

plt.xlabel('Año', fontsize=12)
plt.ylabel('Número de Empresas', fontsize=12)
plt.title('Evolución del Número de Empresas en Cataluña por Provincia', fontsize=14)

plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.')))

plt.legend(title='Provincia')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
dataframe_variacio_PIB_per_capita = pd.read_excel('data/Variació PIB per capita.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_per_capita: ", dataframe_variacio_PIB_per_capita.shape)

print(dataframe_variacio_PIB_per_capita.head(5))


In [ ]:
dataframe_variacio_PIB_percentatge = pd.read_excel('data/Variació PIB porcentaje.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_percentatge: ", dataframe_variacio_PIB_percentatge.shape)

print(dataframe_variacio_PIB_percentatge.head(5))

In [ ]:
dataframe_variacio_PIB = pd.read_excel('data/Variació PIB.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_percentatge: ", dataframe_variacio_PIB_percentatge.shape)

print(dataframe_variacio_PIB_percentatge.head(5))

Donats els gràfics i les sèries disponibles sobre factors clau a Espanya —com ara el PIB i el creixement de la població, que mostren una distribució desigual—, es conclou que el focus d’anàlisi amb més potencial es troba a Catalunya. Es tracta d’una de les zones més poblades, amb dinàmica de creixement i una elevada concentració d’entitats rellevants per al PIB.

A més, es troba que a Catalunya, i segons dades internes de Caixa d’Enginyers, la distribució de socis per regió és la següent:

**⚠️ WARNING:** Distribució ficticia de socis per regions (Temporal)

| Regions              | Socis | % sobre total | Variació interanual | Nota breu                                         |
|---------------------------------|-------:|--------------:|---------------------:|----------------------------------------------------|
| Àrea Metropolitana de Barcelona | 72.000 | 60,0%         | +3,2%                | Alta concentració urbana i hubs tecnològics      |
| Comarques Gironines            | 12.600 | 10,5%         | +4,1%                | Creixement en renovables i turisme de qualitat     |
| Camp de Tarragona              | 10.800 | 9,0%          | +3,7%                | Indústria química i logística portuaria            |
| Penedès                        | 8.400  | 7,0%          | +3,5%                | Agroalimentari i teixit exportador                |
| Comarques Centrals             | 7.200  | 6,0%          | +2,8%                | Pymes industrials diversificades                  |
| Ponent                         | 4.800  | 4,0%          | +2,4%                | Agroindústria i energia                            |
| Terres de l’Ebre               | 2.400  | 2,0%          | +2,0%                | Transició energètica i serveis                  |
| Alt Pirineu i Aran             | 1.800  | 1,5%          | +1,7%                | Turisme sostenible i economia de montanya           |
| **Total**                      | **120.000** | **100%**   | —                    | —                                                  |

